In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

from geometry_msgs.msg import PoseStamped

from tf.transformations import euler_from_quaternion, quaternion_from_euler
import numpy as np

In [2]:
print "============ Starting tutorial setup"
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial',
                anonymous=True)

============ Starting tutorial setup


In [3]:
robot = moveit_commander.RobotCommander()

In [4]:
robot.get_group_names()

['endeffector', 'manipulator']

In [5]:
robot.get_joint_names()

['ASSUMED_FIXED_ROOT_JOINT',
 'world_joint',
 'base_link-base_fixed_joint',
 'shoulder_pan_joint',
 'shoulder_lift_joint',
 'elbow_joint',
 'wrist_1_joint',
 'wrist_2_joint',
 'wrist_3_joint',
 'ee_fixed_joint',
 'wrist_3_link-tool0_fixed_joint']

In [6]:
scene = moveit_commander.PlanningSceneInterface()
group = moveit_commander.MoveGroupCommander("manipulator")
scene = moveit_commander.PlanningSceneInterface()
robot = moveit_commander.RobotCommander()
print robot.get_planning_frame()


'/world'

header: 
  seq: 0
  stamp: 
    secs: 1508581436
    nsecs: 869122028
  frame_id: /world
pose: 
  position: 
    x: -0.00713136303917
    y: 0.668372222968
    z: 0.0983183781403
  orientation: 
    x: -0.0335742420499
    y: 0.693696777508
    z: 0.0608308028075
    w: 0.716908058663


x: 0.610161194149
y: 0.251941120783
z: -0.668951362971
w: 0.341662205683

In [37]:
print "start_pose.orientation.x =", pose.pose.orientation.x, ";"
print "start_pose.orientation.y =", pose.pose.orientation.y, ";"
print "start_pose.orientation.z =", pose.pose.orientation.z, ";"
print "start_pose.orientation.w =", pose.pose.orientation.w, ";"

print "start_pose.position.x =", pose.pose.position.x, ";"
print "start_pose.position.y =", pose.pose.position.y, ";"
print "start_pose.position.z =", pose.pose.position.z, ";"


start_pose.orientation.x = 0.610161194149 ;
start_pose.orientation.y = 0.251941120783 ;
start_pose.orientation.z = -0.668951362971 ;
start_pose.orientation.w = 0.341662205683 ;
start_pose.position.x = -0.204612917304 ;
start_pose.position.y = -0.584701611682 ;
start_pose.position.z = 0.0517568746012 ;


In [42]:
zeroPose = group.get_current_pose().pose
zeroPose.position.x = 0
zeroPose.position.y = 0.5
zeroPose.position.z = 0.1

zeroPose.orientation.x = 0.5
zeroPose.orientation.y = 0.5
zeroPose.orientation.z = -0.5
zeroPose.orientation.w = 0.5

initialJoinValues = np.radians([77.32632695,  -69.73220526,  121.29976102, -141.59751593, -90.06095192,  168.36293361])

def goToPose(pose):
    waypoints = []
    waypoints.append(pose)
    (plan, fraction) = group.compute_cartesian_path(
                         waypoints,   # waypoints to follow
                         0.005,        # eef_step
                         0.0)         # jump_threshold
    print fraction
    if fraction == 1.0: 
        plan = group.retime_trajectory(robot.get_current_state(), plan, 0.08)
        group.execute(plan)


In [39]:
def pulsaBoton(waypoints, i, tol = 0.023):
    wpose = geometry_msgs.msg.Pose()
    wpose.orientation.x = 0.5
    wpose.orientation.y = 0.5
    wpose.orientation.z = -0.5
    wpose.orientation.w = 0.5
    wpose.position.x = waypoints[0].position.x + (0.07 * i)
    wpose.position.y = waypoints[0].position.y
    wpose.position.z = waypoints[0].position.z
    waypoints.append(copy.deepcopy(wpose))
    wpose.position.z = wpose.position.z - (tol/2)
    waypoints.append(copy.deepcopy(wpose))
    wpose.position.z = wpose.position.z - (tol/2)
    waypoints.append(copy.deepcopy(wpose))
    wpose.position.z = wpose.position.z + (tol/2)
    waypoints.append(copy.deepcopy(wpose))
    wpose.position.z = wpose.position.z + (tol/2)
    waypoints.append(copy.deepcopy(wpose))



In [40]:
goToPose(zeroPose)

1.0


In [32]:
state = robot.get_current_state()

In [36]:
np.degrees(state.joint_state.position)

array([  77.32632695,  -69.73220526,  121.29976102, -141.59751593,
        -90.06095192,  168.36293361])

In [36]:
waypoints = []
waypoints.append(zeroPose)
pulsaBoton(waypoints, 1)
pulsaBoton(waypoints, 5)


In [38]:
group.set_max_velocity_scaling_factor(0.05)
group.set_max_acceleration_scaling_factor(0.05)

(plan, fraction) = group.compute_cartesian_path(
                             waypoints,   # waypoints to follow
                             0.01,        # eef_step
                             0.00)         # jump_threshold
plan = group.retime_trajectory(robot.get_current_state(), plan, 2)

print fraction

1.0


In [39]:
group.execute(plan)

True

In [25]:
goToZero()

0.370044052863


In [434]:
import random
listB = range(6)
random.shuffle(listB )

waypoints = []
waypoints.append(zeroPose)
listB = range(6)
random.shuffle(listB )
for i in listB:
    pulsaBoton(waypoints, i)
(plan, fraction) = group.compute_cartesian_path(
                         waypoints,   # waypoints to follow
                         0.005,        # eef_step
                         0.0)         # jump_threshold
print "Fraction: ", fraction

plan = group.retime_trajectory(robot.get_current_state(), plan, 0.08)
group.execute(plan)




Fraction:  0.0


False

In [18]:
f = open("geometria.txt", 'r')
lines = f.readlines()
points = []
for l in lines:
    if "Punto en (" in l:
        values = l.split("(")[1].split(")")[0].split(",")[0:2]
        points.append(map(lambda x: float(x) / 10.0, values))
print "Total: ", len(points)       

Total:  36


In [23]:
group.allow_replanning(True)
group.allow_looking(True)

def goToPoint(i, execute=True):
    waypoints = []
    pose =  group.get_current_pose().pose
    waypoints.append(pose)
    wpose = geometry_msgs.msg.Pose()
    wpose.orientation.x = 0.5
    wpose.orientation.y = 0.5
    wpose.orientation.z = -0.5
    wpose.orientation.w = 0.5
    wpose.position.x = points[i][0] / 100
    wpose.position.y = .points[i][1] / 100
    wpose.position.z = 0.1
    waypoints.append(copy.deepcopy(wpose))
    
    (plan, fraction) = group.compute_cartesian_path(
                         waypoints,   # waypoints to follow
                         0.1,        # eef_step
                         0.0000, False)         # jump_threshold
    print "Fraction: ", fraction
    plan = group.retime_trajectory(robot.get_current_state(), plan, 0.2)
    if execute and fraction == 1.0:
        group.execute(plan)

In [24]:
#for i in range(36):
#    goToPoint(i)

goToPoint(18)

Fraction:  1.0


In [139]:
zeroPose = group.get_current_pose().pose


In [140]:
zeroPose

position: 
  x: 0.0371944240549
  y: -0.477343383423
  z: 0.33158252199
orientation: 
  x: -0.519799578703
  y: 0.452835065289
  z: 0.572651180878
  w: 0.443643355257

In [138]:
group.get_current_pose().pose

position: 
  x: -0.470271979396
  y: -0.694113931605
  z: 0.133951472553
orientation: 
  x: -0.429213022331
  y: 0.595713730535
  z: 0.350665661216
  w: 0.581321706766

In [379]:
points[-1][1]

-4.5430399999999995

In [96]:
def fromTo(i, j, execute):
    tol = 0.
    waypoints = []
    sleep(0.1)
    pose =  group.get_current_pose().pose
    wpose = geometry_msgs.msg.Pose()
    wpose.orientation.x = 0.5
    wpose.orientation.y = 0.5
    wpose.orientation.z = -0.5
    wpose.orientation.w = 0.5
    wpose.position.x = points[i][0] / 100
    wpose.position.y = points[i][1] / 100
    wpose.position.z = pose.position.z
    waypoints.append(copy.deepcopy(wpose))
    wpose.position.z = wpose.position.z - (tol/2)
     waypoints.append(copy.deepcopy(wpose))
    
    if(abs(i - j) >= 23):
        wpose.position.x = points[17][0] / 100
        wpose.position.y = points[17][1] / 100
        waypoints.append(copy.deepcopy(wpose))
    
    wpose.position.x = points[j][0] / 100
    wpose.position.y = points[j][1] / 100
    waypoints.append(copy.deepcopy(wpose))
    
    wpose.position.x = points[17][0] / 100
    wpose.position.y = points[17][1] / 100
    waypoints.append(copy.deepcopy(wpose))
    
    (plan, fraction) = group.compute_cartesian_path(
                         waypoints,   # waypoints to follow
                         0.01,        # eef_step
                         0.0, False)         # jump_threshold
    if fraction != 1.0:
        print "Fraction: ", i ,j, fraction
    
    plan = group.retime_trajectory(robot.get_current_state(), plan, 0.7)
    if execute:
        print group.execute(plan)

In [100]:
from random import randint
from time import sleep
group.set_goal_joint_tolerance(0.1)
for i in range(10):
    fromTo(randint(0, 35), randint(0, 35), True)


True
Fraction:  1 23 0.0768115942029
False
False
False
False
False
False
Fraction:  2 28 0.0721088435374
False
False
False


In [82]:
group.get_current_rpy()

[-2.8354030634862633, 1.570422923183557, 1.876991917288172]

In [108]:
group.set_rpy_target([0.6, 1.4200971760862233, -1.157352193535225])

In [109]:
a = group.plan()

In [110]:
group.execute(a)

False

In [13]:
goToZero()

In [15]:
zeroPose

position: 
  x: -0.13462661706
  y: -0.584128417079
  z: 0.0943074019731
orientation: 
  x: 0.610178881466
  y: 0.25194157244
  z: -0.668948832894
  w: 0.34163523773

In [37]:
q = group.get_current_pose().pose.orientation
np.degrees(euler_from_quaternion((q.x, q.y, q.z, q.w)))
d = np.radians([0, 90, -90])
quaternion_from_euler(d[0], d[1], d[2])

array([ 0.5,  0.5, -0.5,  0.5])

In [21]:
euler_from_quaternion(q)

TypeError: 'Quaternion' object has no attribute '__getitem__'

array([ 31.87242469,  81.30274756, -98.34043792])

In [16]:
def push(waypoints, i, bPush = True, tol = 0.02):
    wpose = geometry_msgs.msg.Pose()
    wpose.orientation.x = 0.5
    wpose.orientation.y = 0.5
    wpose.orientation.z = -0.5
    wpose.orientation.w = 0.5
    wpose.position.x = points[i][0] / 100
    wpose.position.y = points[i][1] / 100
    wpose.position.z = waypoints[0].position.z
    waypoints.append(copy.deepcopy(wpose))
    
    if(bPush):
        wpose.position.z = wpose.position.z - tol
        waypoints.append(copy.deepcopy(wpose))

        wpose.position.z = wpose.position.z + tol
        waypoints.append(copy.deepcopy(wpose))

In [18]:
zeroPose = group.get_current_pose().pose
zeroPose.orientation.x = 0.5
zeroPose.orientation.y = 0.5
zeroPose.orientation.z = -0.5
zeroPose.orientation.w = 0.5

In [19]:
unoPose = group.get_current_pose().pose
unoPose.orientation.x = 0.5
unoPose.orientation.y = 0.5
unoPose.orientation.z = -0.5
unoPose.orientation.w = 0.5

In [137]:
f = open("geometria.txt", 'r')
lines = f.readlines()
points = []
for l in lines:
    if "Punto en (" in l:
        values = l.split("(")[1].split(")")[0].split(",")[0:2]
        points.append(map(lambda x: float(x) / 10.0, values))
print "Total: ", len(points)      

Total:  36


In [39]:
waypoints = []
waypoints.append(zeroPose)


#for i in range(10):
#    push(waypoints, current)
#    nextButton = randint(0, 35)
#    if abs(current - nextButton) > 20:
#        push(waypoints, 18, bPush = False)
#    current = nextButton
     
for i in range(0, 36):
    push(waypoints, i, tol = 0.02)
for i in range(0, 36):
    push(waypoints, 35 - i,  tol = 0.020)

group.set_max_acceleration_scaling_factor(1)
group.set_max_velocity_scaling_factor(1)
(plan, fraction) = group.compute_cartesian_path(
                     waypoints,   # waypoints to follow
                     0.01,        # eef_step
                     0.0)         # jump_threshold

print fraction
plan = group.retime_trajectory(robot.get_current_state(), plan, 1)

if fraction == 1.0:
    group.execute(plan)

1.0


[]

In [21]:
unoPose

position: 
  x: 0.260488871747
  y: -0.509645182819
  z: 0.187115384785
orientation: 
  x: 0.5
  y: 0.5
  z: -0.5
  w: 0.5

In [23]:
waypoints = []
waypoints.append(unoPose)
waypoints.append(zeroPose)
waypoints.append(unoPose)

In [56]:
group.allow_replanning(True)
group.allow_looking(True)
group.set_goal_position_tolerance(0.1)

group.se

(plan, fraction) = group.compute_cartesian_path(
                     waypoints,   # waypoints to follow
                     0.001,        # eef_step
                     0.)         # jump_threshold
print fraction

0.422977846471


In [26]:
group.execute(plan)

True

In [38]:

p = PoseStamped()
p.header.frame_id = robot.get_planning_frame()
p.pose.position.x = 0.
p.pose.position.y = -0.8
p.pose.position.z = 0.1
scene.add_box("obstacle", p, (0.2, 0.4, 0.2))

scene.remove_world_object("obstacle")

p = PoseStamped()
p.header.frame_id = robot.get_planning_frame()
p.pose.position.x = 0
p.pose.position.y = 0
p.pose.position.z = -0.12
scene.add_box("floor", p, (2, 2, 0.2))


p = PoseStamped()
p.header.frame_id = robot.get_planning_frame()
p.pose.position.x = 0
p.pose.position.y = 0.4
p.pose.position.z = 0
scene.add_box("wall", p, (2, 0.2, 2))

p = PoseStamped()
p.header.frame_id = robot.get_planning_frame()
p = robot.get_link("ee_link").pose()
p.pose.position.z -= 0.043
scene.attach_box(robot.get_link("ee_link").name(), "box", p, size= (0.086, 0.05, 0.05))

In [157]:
scene.remove_world_object("floor")
scene.remove_world_object("wall")
scene.remove_attached_object("box")

In [176]:

scene.remove_attached_object("box")

In [34]:
scene.get_attached_objects()

{'box': link_name: ee_link
 object: 
   header: 
     seq: 0
     stamp: 
       secs: 0
       nsecs:         0
     frame_id: ee_link
   id: box
   type: 
     key: ''
     db: ''
   primitives: 
     - 
       type: 1
       dimensions: [0.086, 0.05, 0.05]
   primitive_poses: 
     - 
       position: 
         x: 0.0429996181517
         y: -7.9817333769e-06
         z: 2.84109435757e-06
       orientation: 
         x: 1.19806912601e-05
         y: 4.05860855123e-06
         z: 7.38760899566e-06
         w: 0.999999999893
   meshes: []
   mesh_poses: []
   planes: []
   plane_poses: []
   operation: 0
 touch_links: ['ee_link']
 detach_posture: 
   header: 
     seq: 0
     stamp: 
       secs: 0
       nsecs:         0
     frame_id: ''
   joint_names: []
   points: []
 weight: 0.0}

Help on method attach_box in module moveit_commander.planning_scene_interface:

attach_box(self, link, name, pose=None, size=(1, 1, 1), touch_links=[]) method of moveit_commander.planning_scene_interface.PlanningSceneInterface instance

